<a href="https://colab.research.google.com/github/kyunghyuncho/ammi-2019-nlp/blob/master/01-day-LM/neural_lm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Language Modeling

In [1]:
import os
import sys
sys.path.append('utils/')

In [2]:
import utils.ngram_utils as ngram_utils
from utils.amazon_dataset import AmazonDataset, pad, batchify
from torch.utils.data import DataLoader
from utils.neural_lm import BagOfNGrams, DecoderMLP, seq2seq
import utils.global_variables as gl
import torch
from tqdm import tqdm_notebook, tqdm
_tqdm = tqdm_notebook
from utils.ngram_utils import NgramLM

In [3]:
torch.manual_seed(1)


In [4]:
# Read data from .txt files and create lists of reviews

train_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_train.txt', 'r') as f:
    train_data = [review for review in f.read().split('\n') if review]
    
valid_data = []
# create a list of all the reviews 
with open('../data/amazon_reviews_clothing_valid.txt', 'r') as f:
    valid_data = [review for review in f.read().split('\n') if review]
    

In [ ]:
len(train_data), len(valid_data)

(222919, 27869)

In [ ]:
train_data = train_data[:100]
valid_data = valid_data[:10]

In [ ]:
# Tokenize the Datasets
# TODO: this takes a really long time !! why?
train_data_tokenized, all_tokens_train = ngram_utils.tokenize_dataset(train_data)
valid_data_tokenized, all_tokens_valid = ngram_utils.tokenize_dataset(valid_data)


100it [00:00, 594.17it/s]
10it [00:00, 2091.92it/s]


In [ ]:
train_ngram_lm = NgramLM(train_data_tokenized, all_tokens_train, n=1)
valid_ngram_lm = NgramLM(valid_data_tokenized, all_tokens_valid, n=1)

In [ ]:
# TODO: make sure this makes sense for the valid data -- shouldn't we use the same dict for it?
train_unigram = train_ngram_lm.ngram_data
train_id2token_unigram = train_ngram_lm.id2token

valid_unigram = valid_ngram_lm.ngram_data
# valid_id2token_unigram = valid_ngram_lm.id2token 

In [ ]:
# TODO: make sure this makes sense for the valid data -- shouldn't we use the same dict for it?
N = 10
train_data_id = train_ngram_lm.create_data_id(train_unigram)
train_data_id_merged = train_ngram_lm.create_data_id_merged(train_data_id, N)

valid_data_id = train_ngram_lm.create_data_id(valid_unigram)
valid_data_id_merged = train_ngram_lm.create_data_id_merged(valid_data_id, N)

In [ ]:
# train_data_id[:2], train_data_id_merged[:2]

In [ ]:
# len(train_data_id_merged), len(valid_data_id_merged), train_data_id_merged[0]

In [ ]:
train_dataset = AmazonDataset(train_data_id_merged, max_inp_length=None, use_cuda=True)
train_loader = DataLoader(train_dataset, batch_size=1048576, collate_fn=batchify, shuffle=True)

valid_dataset = AmazonDataset(valid_data_id_merged, max_inp_length=None, use_cuda=True)
valid_loader = DataLoader(valid_dataset, batch_size=1048576, collate_fn=batchify, shuffle=True)

100%|██████████| 459/459 [00:00<00:00, 15177.11it/s]


In [ ]:
# train_dataset, train_loader

In [ ]:
# train_data_id_merged[:5]

In [ ]:
# train_dataset[0], train_dataset[0][0].shape, \
# valid_dataset[0], valid_dataset[0][0].shape

In [ ]:
num_epochs = 500
encoder = BagOfNGrams(len(train_id2token_unigram), emb_dim=300, hidden_size=512, out_size=256, activation='Tanh', nlayers=1, reduce='mean', dropout=0.0, batch_norm=False)
decoder = DecoderMLP(input_size=256, output_size=len(train_id2token_unigram), hidden_size=512)
model = seq2seq(encoder, decoder, train_id2token_unigram, use_cuda=False, lr=1e-1)

for epoch in range(num_epochs):
    # Train
    train_loss_epoch = 0
    for i, (data, labels) in tqdm(enumerate(train_loader), disable=True):
        prediction, loss = model.train_step(data, labels)
        train_loss_epoch += loss
    if epoch % 1 == 0:
        print("Epoch {}: Loss {}".format(epoch, train_loss_epoch))
#         print("prediction ", prediction)

        generated = model.eval_step()
        generated_str = ' '.join([g[0] for g in generated])
        print("Generated Sentence: ", generated_str)
        print("")
        


Epoch 0: Loss 7.605658054351807
Generated Sentence:  13 weekends weekends stash stash boring giving high high high purse purse holding participation finish finish corsican corsican corsican limited

Epoch 1: Loss 7.593083381652832
Generated Sentence:  13 weekends weekends stash stash boring giving high high high purse purse holding participation finish finish corsican corsican corsican limited

Epoch 2: Loss 7.5744218826293945
Generated Sentence:  13 weekends weekends stash ' ' ' boring times low sets limited sets considers considers considers you you you you

Epoch 3: Loss 7.54962158203125
Generated Sentence:  13 weekends you you you pronouce pronouce pronouce lesser lesser excited . lesser lesser thrilled and you you you you

Epoch 4: Loss 7.518049240112305
Generated Sentence:  you you you to , , , . . . . . . . . . . . . .

Epoch 5: Loss 7.478518009185791
Generated Sentence:  . . . . . . . . . . . . . . . . . . . .

Epoch 6: Loss 7.428413391113281
Generated Sentence:  . . . . . . . 